In [68]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import datetime
from datetime import datetime

In [69]:
# Action Network URL.
nba_url = 'https://www.actionnetwork.com/nba/odds'

# Takes the URL and makes it an html - takes in a string (URL) and returns
# the html in string form.
def get_url(url):
  html_str = requests.get(url).text
  return html_str

In [70]:
# Stores the HTML string in a variable.
html = get_url(nba_url)

In [71]:
# Creates a list to store the scraped Odds.
odds = []

In [72]:
# Starts to Scrape using BeautifulSoup
soup = BeautifulSoup(html)
odds_soup = soup.find_all('div', class_='custom-1u3dgby e4ukmj14')

In [73]:
odds_soup

[<div class="custom-1u3dgby e4ukmj14"><table><thead><tr><th>Scheduled</th><th>Open</th><th class="custom-1ui3ly9 ebi37p0"><a class="" href="https://switchboard.actionnetwork.com/offers?bookId=68&amp;stateCode=world&amp;context=web-compareodds-banner&amp;deviceId="><div class="compare-odds-book-header-cell__logo"><img alt="DraftKings" class="" height="21" src="https://assets.actionnetwork.com/477013_DraftKings@1x.png"/></div></a></th><th class="custom-1ui3ly9 ebi37p0"><a class="" href="https://switchboard.actionnetwork.com/offers?bookId=75&amp;stateCode=world&amp;context=web-compareodds-banner&amp;deviceId="><div class="compare-odds-book-header-cell__logo"><img alt="BetMGM NJ" class="" height="21" src="https://assets.actionnetwork.com/779359_BetMGM800x200@1x.png"/></div></a></th><th class="custom-1ui3ly9 ebi37p0"><a class="" href="https://switchboard.actionnetwork.com/offers?bookId=69&amp;stateCode=world&amp;context=web-compareodds-banner&amp;deviceId="><div class="compare-odds-book-hea

In [95]:
def nba_odds(html):
    
    odds = []
    
    # Finds all the TR tags in the soup_table
    soup_table = odds_soup[0].find_all('tr')
    
    # The list of the bookies
    bookie_list = ['Draft Kings', 'Bet MGM', 'Fanduel', 'PointsBet', 'Sugar House', 'Bet365', 'Unibet', 'Caesar' + "'s " + 'Sportsbook']
    
    # Sets the index to 1
    ind = 1
    
    for game in soup_table[1:]:
        # Create dictionaries for odds of each team
        team1_odds = dict()
        team2_odds = dict()
        
        # Grabs the div tag with the specified class
        team_names = game.find_all('div', class_='game-info__team')
        
        # Grabs the team names from the html
        name1 = team_names[0].find(class_='game-info__team--desktop').text
        name2 = team_names[1].find(class_='game-info__team--desktop').text
        
        # Sets the Team Names in the Dictionaries
        team1_odds['Team'] = name1
        team2_odds['Team'] = name2
        
        # Access the tag for game odds
        game_layer1 = game.find_all('td', class_='best-odds__last-type-cell')
        
        # Sets the index to 0
        index = 0
        
        # Loops through the odds by each bookie
        for bookie in game_layer1:
            odds_team1 = bookie.find_all('span', class_='highlight-text__children')[0].text
            odds_team2 = bookie.find_all('span', class_='highlight-text__children')[2].text
        
            # Assigns the Name to the Dictionary
            bookie = bookie_list[index]
            team1_odds[bookie] = odds_team1
            team2_odds[bookie] = odds_team2
            
            index += 1
            
        ind += 1
        
        # Appends the odds information into the odds list
        odds.append(team1_odds)
        odds.append(team2_odds)
        
    # Instantiates the DataFrame of Soccer Odds
    nba_odds_df = pd.DataFrame(odds)
    pd.set_option('display.max_columns', 500)

    # Sends the odds to an excel sheet in a specified folder (This folder can be changed to wherever you want)
    dt = datetime.now()
    writer = pd.ExcelWriter(r'/Users/sarahcasale/Documents/Betting Scraper/' + dt.strftime('%b-%d-%Y') + '_nba_data' + '.xlsx')
    nba_odds_df.to_excel(writer, sheet_name='NBA', index=False, na_rep='NaN')
    
    # Auto-adjust columns' width
    for column in nba_odds_df:
        column_width = max(nba_odds_df[column].astype(str).map(len).max(), len(column))
        col_idx = nba_odds_df.columns.get_loc(column)
        writer.sheets['NBA'].set_column(col_idx, col_idx, column_width)
    
    writer.save()
    
    return nba_odds_df

In [96]:
nba_odds_df = nba_odds(html)

In [97]:
nba_odds_df

,Team,Draft Kings,Bet MGM,Fanduel,PointsBet,Sugar House,Bet365,Unibet,Caesar's Sportsbook
0,Hawks,+9,+7.5,+9,+8.5,+8.5,+8.5,+8.5,+8.5
1,Jazz,-9,-7.5,-9,-8.5,-8.5,-8.5,-8.5,-8.5
2,Trail Blazers,+3,+2.5,+3,+3,+3.5,+3,+3.5,+3
3,Clippers,-3,-2.5,-3,-3,-3.5,-3,-3.5,-3
4,Bucks,-7,-7,-7,-6.5,-7,-7,-7,-6.5
5,76ers,+7,+7,+7,+6.5,+7,+7,+7,+6.5
